# Iteration & Generation

***
## Iterators and Iterability
You have surely had some exposure by now to Python's capacity for iteration through sequences (such as string, lists, etc.) and the ability for programmers to define new, customized classes. You may have not, however, been familiarized with the structures that enable you to make your own classes iterable.

For an object in Python to be iterable, it must satisfy the "Iterator Protocol," which entails having two particular methods:
- Object.\_\_iter\_\_( ), a function that returns an iterator
- Object.\_\_next\_\_( ), a function that controls the iteration process, as well as its eventual termination 

For example, let's build our own iterable class called VowelCatcher. VowelCatcher will take in a string on initialization and, when iterated upon, will return "T" if the corresponding letter in the given string is a vowel, returning "F" otherwise. At the end of iteration, it will also print off the vowel count and a converted string of T/F.

In [1]:
class VowelCatcher():
    def __init__(self, given_string):
        self.given_string = given_string
        self.vowel_count = 0
        self.converted = ""
        # An iterator has to be able to "remember" where it is as it progresses
        if(len(given_string)>=1):
            self.position = 0
    
    def __iter__(self):
        """All __iter__ has to do is to return itself"""
        return(self)
    
    def __next__(self):
        # Checks position to ensure it appropriately signals end-of-iteration upon conclusion
        # This prevents errors from going out-of-scope
        if(self.position == len(self.given_string)):
            print("Vowel Count: "+str(self.vowel_count))
            print("Converted: "+str(self.converted))
            raise(StopIteration)
        else:
            self.position += 1
            #To simply print the letters directly: return(self.given_string[self.position-1])
            if(self.given_string[self.position-1] in ["A","a","E","e","I","i","O","o","U","u"]):
                self.vowel_count += 1
                self.converted += "T"
                return("T")
            else:
                self.converted += "F"
                return("F")

In [2]:
# Example of iteration through an instance of our new iterable class
for i in VowelCatcher(given_string="Lobster"):
    print(i)

F
T
F
F
F
T
F
Vowel Count: 2
Converted: FTFFFTF


### next(Object) and Iterator Exhaustion
Let's declare a new instance of our iterable object class to demonstrate some properties of iterators / iteration that are worth knowing:
- We can manually advance an iterator with next(Iterator)
- Iterators only move forward and are non-reusable (but you can redeclare them as needed)

Below are examples of advancing through an iterator using: next(obj), a for-loop, and both next(obj) and a for-loop.

In [182]:
VC_Instance = VowelCatcher("Kayak")
print(next(VC_Instance))
print(next(VC_Instance))
print(next(VC_Instance))
print(next(VC_Instance))
print(next(VC_Instance))
# Running the line below triggers a now-exhausted iterator, triggering StopIteration
# print(next(VC_Iterator))

F
T
F
T
F


In [183]:
# Note how the for-loop will iterate until StopIteration is raised
# and then quietly terminate the loop
VC_Instance = VowelCatcher("Kayak")
for i in VC_Instance:
    print(i)
    

F
T
F
T
F
Vowel Count: 2
Converted: FTFTF


In [187]:
# Note the behavior of the for-loop after iterator engaged by next(Obj)
VC_Instance = VowelCatcher("Kayak")
next(VC_Instance)
next(VC_Instance)
for i in VC_Instance:
    print(i)

F
T
F
Vowel Count: 2
Converted: FTFTF


***
# Generators
In the simplest form, a generator is any function that uses "yield", which returns from a function, but freezes the function until called to resume execution, rather than terminating the function the way a return statement does.

The process of usinng a generator is effectively:
- The function is called and returns an iterable generator object
- next(generator_variable) is called and the function executes until its first yield statement
- the function freezes and will resume when next(generator_variable) is called again

For example, if we wanted to have a function that returned airport codes, but the rest of our program doesn't necessarily need all of the codes at the same time, and taking one at a time would suffice, a generator could be beneficial. Using a generator would avoid having to wait for the function to process all of the airport codes and having to store them all at once; a function could be written to simply "yield" one airport code each time it was called.

In [244]:
def Airports():
    for airport_code in ['LGA', 'DTW', 'SFO', 'ATL', 'JFK', 'DEN']:
        yield(airport_code)

In [249]:
airport_gen = Airports()
print(airport_gen)

<generator object Airports at 0x110c0e728>


In [251]:
airport_gen = Airports()
print(next(airport_gen), next(airport_gen), next(airport_gen))
print("Go Blue!")
print(next(airport_gen), next(airport_gen))

LGA DTW SFO
Go Blue!
ATL JFK


In [254]:
# An example of generator use in a function
def FlightAnnouncer(n):
    airport_stream = Airports()
    for _ in range(n):
        flight_from = next(airport_stream)
        flight_to = next(airport_stream)
        print(f"Our next flight is from {flight_from} to {flight_to}")
FlightAnnouncer(n=3)

Our next flight is from LGA to DTW
Our next flight is from SFO to ATL
Our next flight is from JFK to DEN


Some additional properties of generators to be aware of:
- Finite generators are non-reusable once exhausted // StopIter raised
- Generators can made to infinitely yield values
- As you can see with the dir() method // dir(gen_object), generators automatically have \__iter__() and \__next__()

In [261]:
print(dir(Airports()))

['__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__next__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']


***

## Generator Expressions
Instead of making function-based generators, Python also affords the ability to create generator expressions.

Like function-based generators, generator expressions are exhaustible and can be progressed with next().

Visually, they may look similar to comprehensions

In [23]:
instruments = ['oboe','clarinet','saxophone','french horn','flute','trumpet','piano']

# List Comprehension
print([(len(each),each) for each in instruments],'\n')

# Dictionary Comprehension
print({u:len(u) for u in instruments},'\n')

# A Generator Expression (rather than a Tuple Comprehension, which it may syntactically look like)
genExp = (len(each) for each in instruments)
print(genExp)
print(next(genExp))

[(4, 'oboe'), (8, 'clarinet'), (9, 'saxophone'), (11, 'french horn'), (5, 'flute'), (7, 'trumpet'), (5, 'piano')] 

{'oboe': 4, 'clarinet': 8, 'saxophone': 9, 'french horn': 11, 'flute': 5, 'trumpet': 7, 'piano': 5} 

<generator object <genexpr> at 0x7fd51c0bd2a0>
4


***
The below code cell exemplifies how a generator expression can modify the source before output, in this case by applying .upper() to each string object in the sequence.

In [24]:
instrum_gen = (inst.upper() for inst in instruments)

for instrument in instrum_gen:
    print(instrument)

OBOE
CLARINET
SAXOPHONE
FRENCH HORN
FLUTE
TRUMPET
PIANO


The below code cell reflects how generator expressions are also capable of using conditional expressions. In this case, the expression only generates values that are upper-case and are not a space or apostrophe character.

In [25]:
Title = "The Hitchhiker's Guide to the Galaxy"
capitals = (char for char in Title if (char.upper()==char and char not in [" ","'"]))
for letter in capitals:
    print(letter)

T
H
G
G
